# Laboratorium 7 - MOwNiT2

In [264]:
import numpy as np

## Iteracyjne metody rozwiązywania równań liniowych


In [265]:
A1 = np.matrix([[2, 1], [6, 8]])
b1 = np.matrix([12, 14]).transpose()

In [266]:
A2 = np.array([[11.0, -2.0, 3.0, 0.0],
              [-2.0, 12.0, -2.0, 4.0],
              [3.0, -2.0, 11.0, -2.0],
              [0.0, 4.0, -2.0, 9.0]])
b2 = np.array([7.0, 26.0, -12.0, 16.0]).transpose()

### Zadanie 1.
Zaimplementować metodę Jacobiego i przetestować jej działanie na paru układach równań. Porównać z metodą eliminacji Gaussa użytą do tych samych równań.

In [267]:
def jacobi_solve(A: np.matrix, b: np.matrix, iterations: int=1000) -> np.matrix:
    n = A.shape[0]
    x = np.zeros(n)
    k = 0
    while k < iterations:
        x1 = np.zeros(n)
        for i in range(n):
            sum = 0
            for j in range(n):
                if j != i:
                    sum +=A[i,j] * x[j]
            x1[i] = (1 / A[i,i]) * (b[i] - sum)
        x = x1
        k += 1
    return x
    pass

In [268]:
np.allclose(jacobi_solve(A1,b1), np.linalg.solve(A1,b1).transpose())

True

In [269]:
%timeit np.linalg.solve(A1,b1).transpose()
%timeit jacobi_solve(A1,b1)

17.4 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
63.8 ms ± 8.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [270]:
np.allclose(jacobi_solve(A2,b2), np.linalg.solve(A2,b2).transpose())

True

In [271]:
%timeit np.linalg.solve(A2,b2).transpose()
%timeit jacobi_solve(A2,b2)

20.6 µs ± 6.58 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
24.5 ms ± 2.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Zadanie 2.
Zaimplementować metodę Gaussa-Seidela i przetestować na tych samych układach równań, co metodę Jacobiego. 

In [272]:
def gauss_seidel_solve(A: np.matrix, b: np.matrix, iterations: int=1000) -> np.matrix:
    n = A.shape[0]
    x = np.zeros(n)
    k = 0
    while k < iterations:
        x1 = np.zeros(n)
        for i in range(n):
            sumL = 0
            sumU = 0
            for j in range(0,i):
                sumL +=A[i,j] * x1[j]
            for j in range(i+1,n):
                sumU +=A[i,j] * x[j]
            x1[i] = (1 / A[i,i]) * (b[i] - sumL - sumU)
        x = x1
        k += 1
    return x                
    pass

In [273]:
np.allclose(gauss_seidel_solve(A1,b1), np.linalg.solve(A1,b1).transpose())

True

In [274]:
%timeit np.linalg.solve(A1,b1).transpose()
%timeit gauss_seidel_solve(A1,b1)

34.4 µs ± 14.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
The slowest run took 5.96 times longer than the fastest. This could mean that an intermediate result is being cached.
196 ms ± 150 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [275]:
np.allclose(gauss_seidel_solve(A2,b2), np.linalg.solve(A2,b2).transpose())

True

In [276]:
%timeit np.linalg.solve(A2,b2).transpose()
%timeit gauss_seidel_solve(A2,b2)

13.9 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
16.4 ms ± 785 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Zadanie 3.
Zaimplementować metodę SOR i przetestować na tych samych układach.

In [277]:
def sor_solve(A: np.matrix, b: np.matrix, iterations: int=1000) -> np.matrix:
    n = A.shape[0]
    x = np.zeros(n)
    omega = 1.44
    k = 0
    while k < iterations:
        x1 = np.zeros(n)
        for i in range(n):
            sumL = 0
            sumU = 0
            for j in range(0,i):
                sumL +=A[i,j] * x1[j]
            for j in range(i+1,n):
                sumU +=A[i,j] * x[j]
            x1[i] = (1 - omega) * x[i] + (omega / A[i,i]) * (b[i] - sumL - sumU)
        x = x1
        k += 1
    return x  
    pass

In [278]:
np.allclose(sor_solve(A1,b1), np.linalg.solve(A1,b1).transpose())

True

In [279]:
%timeit np.linalg.solve(A1,b1).transpose()
%timeit sor_solve(A1,b1)

18.2 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
120 ms ± 36.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [280]:
np.allclose(sor_solve(A2,b2), np.linalg.solve(A2,b2).transpose())

True

In [281]:
%timeit np.linalg.solve(A2,b2).transpose()
%timeit sor_solve(A2,b2)

16 µs ± 1.9 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
16.9 ms ± 3.06 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Zadanie 4.
Dla powyższych metod porównać tempo zbiegania do rozwiązania.

In [282]:
def check_method(A: np.matrix, b: np.matrix, method, method_name):
    np_solve = np.linalg.solve(A, b)
    for i in range(1,100):
        x = method(A, b, i)
        if(np.allclose(np_solve.transpose(), x) == True):
            print(method_name, ": accurate after", i, "iterations")
            break
    pass

In [283]:
check_method(A1, b1, jacobi_solve, "jacobi_solve")
check_method(A1, b1, gauss_seidel_solve, "gauss_seidel_solve")
check_method(A1, b1, sor_solve, "sor_solve")

jacobi_solve : accurate after 24 iterations
gauss_seidel_solve : accurate after 12 iterations
sor_solve : accurate after 15 iterations


In [284]:
check_method(A2, b2, jacobi_solve, "jacobi_solve")
check_method(A2, b2, gauss_seidel_solve, "gauss_seidel_solve")
check_method(A2, b2, sor_solve, "sor_solve")

jacobi_solve : accurate after 25 iterations
gauss_seidel_solve : accurate after 8 iterations
sor_solve : accurate after 17 iterations
